In [9]:
from datasets import load_from_disk, load_dataset
import pandas as pd
from tqdm import tqdm

In [2]:
mrc_datasets = load_from_disk('./data/train_dataset/')
mrc_train_dataset = mrc_datasets['train']
mrc_valid_dataset = mrc_datasets['validation']

In [3]:
# train data
_id = []  # 중복 확인용
doc_id = []
title = []
context = []
question = []
answer = []
answer_start = []
answer_len = []
question_len = []
context_len = []

for train_data in mrc_train_dataset:
    _id.append(train_data['id'])
    doc_id.append(train_data['document_id'])
    title.append(train_data['title'])
    context.append(train_data['context'])
    question.append(train_data['question'])
    answer.append(train_data['answers']['text'][0])
    context_len.append(len(train_data['context']))
    question_len.append(len(train_data['question']))
    #answer_len.append(answer_len_f(train_data['answers']['text']))
    answer_len.append(len(train_data['answers']['text'][0]))
    answer_start.append(train_data['answers']['answer_start'][0])
    
train_dict = {
    "id":_id,
    "doc_id":doc_id,
    "title":title,
    "context":context,
    "question":question,
    "answer":answer,
    "context_len":context_len,
    "question_len":question_len,
    "answer_len":answer_len,
    "answer_start":answer_start,
}

train_df = pd.DataFrame(train_dict)

# valid data
_id = []  # 중복 확인용
doc_id = []
title = []
context = []
question = []
answer = []
answer_start = []
answer_len = []
question_len = []
context_len = []

for valid_data in mrc_valid_dataset:
    _id.append(valid_data['id'])
    doc_id.append(valid_data['document_id'])
    title.append(valid_data['title'])
    context.append(valid_data['context'])
    question.append(valid_data['question'])
    answer.append(valid_data['answers']['text'][0])
    context_len.append(len(valid_data['context']))
    question_len.append(len(valid_data['question']))
    #answer_len.append(answer_len_f(valid_data['answers']['text']))
    answer_len.append(len(valid_data['answers']['text'][0]))
    answer_start.append(valid_data['answers']['answer_start'][0])
    
valid_dict = {
    "id":_id,
    "doc_id":doc_id,
    "title":title,
    "context":context,
    "question":question,
    "answer":answer,
    "context_len":context_len,
    "question_len":question_len,
    "answer_len":answer_len,
    "answer_start":answer_start,
}

valid_df = pd.DataFrame(valid_dict)

In [4]:
all_df = pd.concat([train_df,valid_df], ignore_index=True)
all_df.head(3)

,id,doc_id,title,context,question,answer,context_len,question_len,answer_len,answer_start
0,mrc-1-000067,18293,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,하원,862,32,2,235
1,mrc-0-004397,51638,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,《경영의 실제》,621,22,8,212
2,mrc-1-000362,5028,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,백성,850,30,2,510


# 정답의 조사를 분석해보자!

## 정답 df 생성

In [17]:
# 정답 df 생성
all_ans_df = all_df['answer']
all_ans_df

0                  하원
1            《경영의 실제》
2                  백성
3                  중국
4                  4개
            ...      
4187             제 양왕
4188    '일곱 개의 신전 광장'
4189           미나미 지로
4190         200,000명
4191             공산주의
Name: answer, Length: 4192, dtype: object

In [25]:
## konlpy
## tweepy를 3.10.0으로 깔아야 합니다.
from konlpy.tag import *
okt=Okt()

In [42]:
pos_cnt = {}
pos_list = {}
origin_list = {}
pos_se_cnt = {} # 맨앞, 맨뒤
for i in tqdm(range(len(all_ans_df))):
    pos = okt.pos(all_ans_df[i])
    # 전체 개수 추가
    for p in pos:
        try:
            pos_cnt[p[1]] += 1
            pos_list[p[1]].append(p[0])
            origin_list[p[1]].append(all_ans_df[i])
        except:
            pos_cnt[p[1]] = 1
            pos_list[p[1]] = [p[0]]
            origin_list[p[1]] = [all_ans_df[i]]
    
    # 맨 앞, 맨 뒤 추가
    try:
        pos_se_cnt[pos[0][1]]+=1
    except:
        pos_se_cnt[pos[0][1]]=1
    if len(pos)>=2:
        try:
            pos_se_cnt[pos[-1][1]]+=1
        except:
            pos_se_cnt[pos[-1][1]]=1

100%|██████████| 4192/4192 [00:01<00:00, 2592.44it/s]


In [37]:
pos_cnt

{'Noun': 7198,
 'Foreign': 257,
 'Josa': 757,
 'Number': 857,
 'Suffix': 232,
 'Verb': 304,
 'Modifier': 218,
 'Punctuation': 527,
 'Alpha': 177,
 'Adverb': 30,
 'Adjective': 116,
 'Determiner': 41,
 'VerbPrefix': 5,
 'Exclamation': 17,
 'KoreanParticle': 1,
 'Conjunction': 2,
 'Eomi': 3}

In [49]:
pos_list['VerbPrefix']

['엇', '안', '잘', '못', '못']

In [50]:
origin_list['VerbPrefix']

['엇박자 맥박이 나오는 부위',
 '안채',
 '헤르만 폰 잘차',
 '샤 자한이 이 무덤을 지을 때 자신이 그녀의 옆에 묻힐 것을 예상하지 못하였기 때문',
 '임신하지 못하는 많은 사람들']

In [39]:
pos_se_cnt

{'Noun': 5016,
 'Foreign': 177,
 'Number': 677,
 'Suffix': 93,
 'Modifier': 100,
 'Punctuation': 288,
 'Alpha': 64,
 'Adverb': 10,
 'Josa': 78,
 'Verb': 87,
 'Adjective': 40,
 'VerbPrefix': 2,
 'Determiner': 15,
 'Exclamation': 7,
 'KoreanParticle': 1,
 'Eomi': 2}